In [1]:
import sys
sys.path.append('..')
from utils import get_search_term, search_radiopaedia, structure_search_result, get_best_result

In [2]:
import pandas as pd
from tqdm import tqdm
# bm25s

In [3]:
queries = pd.read_csv("queries.csv")

In [4]:
results = []

for query in tqdm(queries["input"].values):
    search_term = get_search_term(query)
    soup = search_radiopaedia(search_term.text)
    all_search_results = [structure_search_result(search_result, i) for i, search_result in enumerate(soup.find_all(class_='search-result'))]
    r = get_best_result(query, all_search_results)
    
    results.append({
        "query": query,
        "search_term": search_term.text,    
        "best_result_id": r.id,
        "best_result_title": all_search_results[r.id]["title"],
    })
    
results = pd.DataFrame(results)
results.to_csv("eval_results1.csv", index=False)

  0%|          | 0/39 [00:00<?, ?it/s]2024-11-16 22:26:41,942 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-11-16 22:26:41,947 - INFO - SearchTerm for query: 'How do I measure tibial tuberosity-trochlear groove (TT-TG) distance on CT?' is: 'tibial tuberosity-trochlear groove (TT-TG) distance'
2024-11-16 22:26:42,748 - INFO - Formatted search results: ID: 0

Title: Tibial tuberosity to trochlear groove distance (TT-TG distance)

Body: The tibial tuberosity to trochlear groove (TT-TG) distance is used to guide the treatment of patellofemoral instability. It can be accurately measured on both CT and MRI although MRI measurements are consistently shorter 6.

Usage

TT-TG and TT-PCL have similar reliability and sensitivity/specif...

2024-11-16 22:26:45,002 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-11-16 22:26:45,004 - INFO - BestSearchResult for query: 'How do I measure tibial tuberosity-trochlear gro

In [12]:
# limit the search results to top 5
results = []

for query in tqdm(queries["input"].values):
    search_term = get_search_term(query)
    soup = search_radiopaedia(search_term.text)
    all_search_results = [structure_search_result(search_result, i) for i, search_result in enumerate(soup.find_all(class_='search-result'))]
    r = get_best_result(query, all_search_results[:5])
    
    results.append({
        "query": query,
        "search_term": search_term.text,    
        "best_result_id": r.id,
        "best_result_title": all_search_results[r.id]["title"],
    })
    
results = pd.DataFrame(results)
results.to_csv("eval_results2.csv", index=False)

  0%|          | 0/39 [00:00<?, ?it/s]2024-11-16 22:34:20,486 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-11-16 22:34:20,495 - INFO - SearchTerm for query: 'How do I measure tibial tuberosity-trochlear groove (TT-TG) distance on CT?' is: 'tibial tuberosity-trochlear groove (TT-TG) distance'
2024-11-16 22:34:21,333 - INFO - Formatted search results: ID: 0

Title: Tibial tuberosity to trochlear groove distance (TT-TG distance)

Body: The tibial tuberosity to trochlear groove (TT-TG) distance is used to guide the treatment of patellofemoral instability. It can be accurately measured on both CT and MRI although MRI measurements are consistently shorter 6.

Usage

TT-TG and TT-PCL have similar reliability and sensitivity/specif...

2024-11-16 22:34:23,657 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2024-11-16 22:34:23,661 - INFO - BestSearchResult for query: 'How do I measure tibial tuberosity-trochlear gro

In [18]:
df.to_csv("search_terms.csv", index=False)

In [1]:
n = 35

query = df.iloc[n]["input"]
soup = search_radiopaedia(df.iloc[n]["search_term"])
all_search_results = [structure_search_result(search_result, i) for i, search_result in enumerate(soup.find_all(class_='search-result'))]
r = get_best_result(df.iloc[n]["input"], all_search_results)
all_search_results[r.id]

NameError: name 'df' is not defined

In [57]:
corpus = [s["body"] for s in all_search_results]
corpus_tokens = bm25s.tokenize(corpus)
retriever = bm25s.BM25(corpus=corpus)
retriever.index(corpus_tokens)
query_tokens = bm25s.tokenize(df.iloc[n]["input"])
docs, scores = retriever.retrieve(query_tokens,k=1)
[s for s in all_search_results if s["body"] == docs[0]][0]

{'id': 4,
 'title': 'Neurocytoma',
 'body': 'Neurocytomas are WHO grade 2\xa0primary CNS neoplasms that derive from neural cells. There are two main types:\n\nintraventricular neurocytoma\n\nmuch more common\n\ncalled a central neurocytoma when located centrally within the lateral ventricles\n\nextraventricular neurocytoma\xa01\n\nBoth types tend to have ...',
 'href': '/articles/neurocytoma?lang=us'}

In [37]:
docs[0], scores[0]

(array(['The Cobb angle\xa0is the most widely used measurement to quantify the magnitude of spinal deformities, especially scoliosis, on plain radiographs. Scoliosis is a lateral spinal curvature with a Cobb angle of >10°\xa04. The Cobb angle technique can also assess the degree of kyphosis or lordosis in the ...'],
       dtype='<U299'),
 array([0.7514223], dtype=float32))